In [14]:
from datasets import Dataset, DatasetDict, load_from_disk
from PIL import Image
import pillow_heif
import requests
import math
import openai
import tqdm

import pandas as pd

import os
import io

from dotenv import load_dotenv

load_dotenv()

client = openai.OpenAI()

In [15]:
df_publicaciones = pd.read_excel('../data/dataset_publicaciones.xlsx')

In [16]:
df_publicaciones = df_publicaciones[df_publicaciones['estado'] == True]

In [17]:
df_publicaciones

,ruta,url,picture,multiple_choice_answer,transcript,estado
0,../data/raw/491453241_18456811006072389_870746...,https://instagram.fblr20-1.fna.fbcdn.net/v/t51...,NaN,En la imagen hay una persona con la cara de un...,Yo cuando me pide espacio,True
1,../data/raw/491442747_18456945862072389_555197...,https://scontent-lga3-2.cdninstagram.com/v/t51...,NaN,Este meme es la pura verdad pal que se ha saca...,Puede que me haya ido como el hoyo en la prueb...,True
2,../data/raw/491973338_18457015648072389_445219...,https://instagram.fbgi1-1.fna.fbcdn.net/v/t51....,NaN,Esta imagen es una talla buena. Muestra a dos ...,Mi bestie y yo montando dos pollas gigantes po...,True
3,../data/raw/491442403_18456919780072389_785138...,https://scontent-iad3-2.cdninstagram.com/v/t51...,NaN,El meme muestra a Chris Pratt con cara de impr...,Yo viendo. Yo viendo como en el examen de mate...,True
4,../data/raw/491336753_18456694879072389_818932...,https://instagram.fdac99-1.fna.fbcdn.net/v/t51...,NaN,En el meme se ve a un papa junto a un montón d...,Ojalá que el nuevo papa sea el papayon,True
...,...,...,...,...,...,...
695,../data/raw/482336003_18448788025072763_161365...,https://instagram.fwzy2-1.fna.fbcdn.net/v/t51....,NaN,Este meme usa a Homero Simpson sentado en la c...,NaN,True
696,../data/raw/481835507_18447965344072763_555599...,https://scontent-mia3-1.cdninstagram.com/v/t51...,NaN,"El meme agarra pal leseo a George Harris, humo...",NaN,True
697,../data/raw/481583836_18447963409072763_234601...,https://instagram.ftpa1-2.fna.fbcdn.net/v/t51....,NaN,"Mira, el meme muestra a Meruane, que es un hum...",NaN,True
698,../data/raw/481473276_18447963808072763_745297...,https://scontent-phx1-1.cdninstagram.com/v/t51...,NaN,Este meme está tirando la talla directa sobre ...,NaN,True


In [18]:
TEMPLATES = [
    "Si hay texto en la imagen, entiende su significado sin repetirlo. Observa también la imagen. Explica qué significa este meme en Chile, usando lenguaje coloquial chileno.",
    "Explica cómo entendería este meme alguien que vive en Chile, usando ejemplos culturales o referencias locales que ayuden a comprender el chiste.",
    "Describe por qué este meme sería gracioso o significativo para un chileno, destacando elementos del contexto social o cotidiano en Chile.",
    "Imagina que le explicas este meme a alguien extranjero: ¿cómo resumirías su sentido y por qué es gracioso para la gente en Chile?"
]

total_cost = 0.0

def generate_aligned_response(base_instruction, base_answer, new_instruction):
    global total_cost

    prompt = (
        f"Prompt base:\n{base_instruction}\n\n"
        f"Respuesta original:\n{base_answer}\n\n"
        f"Tarea:\nReescribe una respuesta alineada a esta nueva instrucción, solo debes devolver la respuesta:\n{new_instruction}"
    )

    rsp = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=300
    )

    usage = rsp.usage
    prompt_tokens = usage.prompt_tokens
    completion_tokens = usage.completion_tokens

    # Costos por millón de tokens (ajusta según tu cuenta)
    cost_per_million_prompt = 2.50 / 1_000_000
    cost_per_million_completion = 10.00 / 1_000_000

    call_cost = (prompt_tokens * cost_per_million_prompt) + (completion_tokens * cost_per_million_completion)
    total_cost += call_cost

    print(f"-> Esta llamada usó {prompt_tokens} prompt tokens + {completion_tokens} completion tokens = ${call_cost:.6f} USD")
    print(f"==> Gasto acumulado: ${total_cost:.6f} USD")

    return rsp.choices[0].message.content.strip()

records = []

for idx, entry in tqdm.tqdm(df_publicaciones.iterrows(), total=len(df_publicaciones)):
    img = Image.open(entry["ruta"])
    url = entry["url"]
    base_answer = entry['multiple_choice_answer']

    # Prompt base → respuesta ya validada
    records.append({
        "image": img,
        "url": url,
        "instruction": TEMPLATES[0],
        "multiple_choice_answer": base_answer,
    })

    # Para cada nueva instrucción, generar respuesta alineada
    for new_instruction in TEMPLATES[1:]:
        aligned_answer = generate_aligned_response(TEMPLATES[0], base_answer, new_instruction)
        records.append({
            "image": img,
            "url": url,
            "instruction": new_instruction,
            "multiple_choice_answer": aligned_answer,
        })

  0%|          | 0/700 [00:00<?, ?it/s]

-> Esta llamada usó 188 prompt tokens + 154 completion tokens = $0.002010 USD
==> Gasto acumulado: $0.002010 USD
-> Esta llamada usó 186 prompt tokens + 145 completion tokens = $0.001915 USD
==> Gasto acumulado: $0.003925 USD


  0%|          | 1/700 [00:07<1:23:03,  7.13s/it]

-> Esta llamada usó 191 prompt tokens + 118 completion tokens = $0.001658 USD
==> Gasto acumulado: $0.005582 USD
-> Esta llamada usó 209 prompt tokens + 139 completion tokens = $0.001913 USD
==> Gasto acumulado: $0.007495 USD
-> Esta llamada usó 207 prompt tokens + 138 completion tokens = $0.001898 USD
==> Gasto acumulado: $0.009392 USD


  0%|          | 2/700 [00:14<1:27:26,  7.52s/it]

...

-> Esta llamada usó 212 prompt tokens + 117 completion tokens = $0.001700 USD
==> Gasto acumulado: $3.998800 USD
-> Esta llamada usó 216 prompt tokens + 148 completion tokens = $0.002020 USD
==> Gasto acumulado: $4.000820 USD
-> Esta llamada usó 214 prompt tokens + 147 completion tokens = $0.002005 USD
==> Gasto acumulado: $4.002825 USD


100%|█████████▉| 699/700 [1:42:40<00:07,  7.52s/it]

-> Esta llamada usó 219 prompt tokens + 136 completion tokens = $0.001908 USD
==> Gasto acumulado: $4.004732 USD
-> Esta llamada usó 216 prompt tokens + 197 completion tokens = $0.002510 USD
==> Gasto acumulado: $4.007242 USD
-> Esta llamada usó 214 prompt tokens + 170 completion tokens = $0.002235 USD
==> Gasto acumulado: $4.009477 USD


100%|██████████| 700/700 [1:42:49<00:00,  8.81s/it]

-> Esta llamada usó 219 prompt tokens + 131 completion tokens = $0.001858 USD
==> Gasto acumulado: $4.011335 USD


In [19]:
records

[{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1069x1112>,
  'url': 'https://instagram.fblr20-1.fna.fbcdn.net/v/t51.2885-15/491453241_18456811006072389_8707465251110731728_n.jpg?stp=dst-jpg_e15_tt6&_nc_ht=instagram.fblr20-1.fna.fbcdn.net&_nc_cat=109&_nc_oc=Q6cZ2QH2AhfyamYlFthVVGups5L1wBpLsK9JVnJWine46BjlMEFxLlqNxtjkrW1RIjkcvV4&_nc_ohc=M5YLE2-qSIUQ7kNvwEvD0dD&_nc_gid=TSJwa3a7RjkxCYPXoW-nfg&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfGwG82g4unVLcEoHHH-tD5J4b5DL29Gw1GH8pGq9aWFxQ&oe=6812ECDD&_nc_sid=10d13b',
  'instruction': 'Si hay texto en la imagen, entiende su significado sin repetirlo. Observa también la imagen. Explica qué significa este meme en Chile, usando lenguaje coloquial chileno.',
  'multiple_choice_answer': 'En la imagen hay una persona con la cara de un planeta y el cuerpo lleno de estrellas y galaxias, como si fuera parte del espacio. Y el texto dice "yo cuando me pide espacio". Es una talla sobre lo de "pedir espacio" en una relación, pero llevado al extremo máxim

In [ ]:
#import tqdm
#
#records = []
#for idx, entry in tqdm.tqdm(df_publicaciones.iterrows()):
#    img = Image.open(entry["ruta"])
#    url = entry["url"]
#    multiple_choice_answer = entry['multiple_choice_answer']
#    transcript = entry['transcript']
#    records.append({
#        "image": img,
#        "url": url,
#        "multiple_choice_answer": multiple_choice_answer,
#        "transcript": transcript
#    })

700it [00:00, 7864.03it/s]


In [23]:
# Limpiamos los nans de la columna 'transcript'
for rec in records:
    if isinstance(rec.get('transcript'), float):
        if math.isnan(rec['transcript']):
            rec['transcript'] = ""
        else:
            rec['transcript'] = str(rec['transcript'])

In [20]:
ds = Dataset.from_list(records)

In [21]:
dataset = DatasetDict({
    "train": ds
})
dataset.save_to_disk("../data/processed/meme_chileno_dataset")

Saving the dataset (2/2 shards): 100%|██████████| 2800/2800 [00:00<00:00, 4748.24 examples/s]


In [22]:
dataset = load_from_disk("../data/processed/meme_chileno_dataset")

In [23]:
dataset['train'][14]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1080x890>,
 'url': 'https://scontent-iad3-2.cdninstagram.com/v/t51.2885-15/491442403_18456919780072389_7851383395431526076_n.jpg?stp=dst-jpg_e35_s1080x1080_sh0.08_tt6&_nc_ht=scontent-iad3-2.cdninstagram.com&_nc_cat=109&_nc_oc=Q6cZ2QEfRI8D-TvwJrwlzW5IKW-r1oges6U76iQujZ5oohpfveqwsR6XwPY6UemHRI0QbXc&_nc_ohc=Viw3U2Kc9eUQ7kNvwE3RB9-&_nc_gid=Kxu2GucTtXcxXcI-NcHLig&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfFw-5hY4UpEntPAIbKontf3w7XxpzsLL2jziyH6EjbK0g&oe=6812E90C&_nc_sid=10d13b',
 'instruction': 'Describe por qué este meme sería gracioso o significativo para un chileno, destacando elementos del contexto social o cotidiano en Chile.',
 'multiple_choice_answer': 'Este meme sería chistoso para un chileno porque refleja esa típica situación en que uno se enfrenta a una prueba y, al principio, parece un regalo del cielo porque hay pocos ejercicios. Pero, en realidad, es una trampa porque esos ejercicios son más difíciles que la cresta. En Chil

In [24]:
dataset.push_to_hub("AndresSebad/memes_instagram_chilenos_es_small")

Uploading the dataset shards: 100%|██████████| 1/1 [02:16<00:00, 136.74s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/AndresSebad/memes_instagram_chilenos_es_small/commit/fb835effd5185dce719c28b51b01cf54992b40d3', commit_message='Upload dataset', commit_description='', oid='fb835effd5185dce719c28b51b01cf54992b40d3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/AndresSebad/memes_instagram_chilenos_es_small', endpoint='https://huggingface.co', repo_type='dataset', repo_id='AndresSebad/memes_instagram_chilenos_es_small'), pr_revision=None, pr_num=None)